# Обучение модели на основе датасета Geo Reviews Dataset 2023

Geo Reviews Dataset 2023 - крупнейший русскоязычный датасет отзывов об организациях, опубликованных на Яндекс Картах ([https://github.com/yandex/geo-reviews-dataset-2023](https://github.com/yandex/geo-reviews-dataset-2023))

## Чтение всех данных из файла формата tskv (Tab-Separated Key-Value)

In [1]:
import pandas as pd

# Функция для чтения строки из датасета
def get_row_from_line(line: str) -> dict[str, str]:

    columns = line.split("\t")

    row: dict[str, str] = dict()
    
    for column in columns:

        delimiter_postion = column.find("=")
        
        column_name = column[:delimiter_postion]
        data = column[delimiter_postion + 1:]
        
        row[column_name] = data

    return row


file_path = r"C:\Users\User\Downloads\geo-reviews-dataset-2023.tskv"

with open(file_path, mode="r", encoding="utf-8") as file:
    rows = [get_row_from_line(line) for line in file]

df = pd.DataFrame(rows)

# Удаляем колонки адреса и названия организации
df = df.drop(["address", "name_ru"], axis=1)

df.head()

,rating,rubrics,text
0,3.,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...
1,5.,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,1.,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,4.,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...
4,5.,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...


## Уменьшение датасета

In [2]:
df = df.loc[:10000]

## Конвертация данных

In [3]:
# Убираем точку из значения рейтинга и конвертируем в int
df["rating"] = pd.to_numeric(df["rating"].str.replace(".", ""))

# Конвертируем рубрики в список
df["rubrics"] = df["rubrics"].apply(lambda x: x.split(";"))

df.head()

,rating,rubrics,text
0,3,[Жилой комплекс],Московский квартал 2.\nШумно : летом по ночам ...
1,5,"[Магазин продуктов, Продукты глубокой заморозк...","Замечательная сеть магазинов в общем, хороший ..."
2,1,[Фитнес-клуб],"Не знаю смутят ли кого-то данные правила, но я..."
3,4,"[Пункт проката, Прокат велосипедов, Сапсёрфинг]",Хорошие условия аренды. \nДружелюбный персонал...
4,5,"[Салон красоты, Визажисты, стилисты, Салон бро...",Топ мастер Ангелина топ во всех смыслах ) Немн...


## Очистка текста отзывов

In [4]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


# Функция для очистки текста
def clean_text(text: str):

    # Удаление переносов строк
    text = text.replace("\\n", "")
    
    # Приведение к нижнему регистру
    text = text.lower()
    
    # Удаление пунктуации и специальных символов
    text = re.sub(r'[^a-zа-яё\s]', '', text)  # Убираем все, кроме букв и пробелов
    
    # Токенизация
    tokens = nltk.word_tokenize(text)
    
    # Удаление стоп-слов
    stop_words = set(stopwords.words('russian'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Лемматизация
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Объединение токенов обратно в строку
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text


# Применение функции очистки к каждому отзыву
df["text"] = df["text"].apply(clean_text)

df.head()

,rating,rubrics,text
0,3,[Жилой комплекс],московский квартал шумно летом ночам дикие гон...
1,5,"[Магазин продуктов, Продукты глубокой заморозк...",замечательная сеть магазинов общем хороший асс...
2,1,[Фитнес-клуб],знаю смутят когото данные правила удивлена хоч...
3,4,"[Пункт проката, Прокат велосипедов, Сапсёрфинг]",хорошие условия аренды дружелюбный персонално ...
4,5,"[Салон красоты, Визажисты, стилисты, Салон бро...",топ мастер ангелина топ смыслах немного волнов...


## Преобразование рубрик в бинарный формат

In [6]:
from sklearn.preprocessing import MultiLabelBinarizer


# Преобразуем рубрики в бинарный формат
mlb = MultiLabelBinarizer()
df_rubrics = pd.DataFrame(mlb.fit_transform(df['rubrics']), columns=mlb.classes_)
df = pd.concat([df, df_rubrics], axis=1)
df = df.drop('rubrics', axis=1)

## Преобразование текста отзыва в векторы TF-IDF

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer


# Преобразуем текст отзыва в векторы TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
text_vectors = vectorizer.fit_transform(df['text'])

## Подготовка данных для обучения

In [8]:
from sklearn.model_selection import train_test_split


# Объединяем векторы TF-IDF с преобразованными рубриками и оценками
X = pd.concat([pd.DataFrame(text_vectors.toarray()), df.drop(['text', 'rating'], axis=1)], axis=1)
y = df['rating']

# Разделяем датасет на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Обучение модели

In [9]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

import neptune
import joblib


# Параметры для обучения моделей
parameters = [
    {"n_estimators": 100,  "learning_rate": 0.2, "max_depth": 5 },
    {"n_estimators": 200,  "learning_rate": 0.1, "max_depth": 10 },
    {"n_estimators": 300,  "learning_rate": 0.05, "max_depth": 15 },
]

for current_parameters in parameters:

    # Инициализация Neptune
    run = neptune.init_run(
        project="svterskov/LSML2",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIyY2FkZDgxNy1kZDJmLTRlMWItOWU0Zi1iZTc5YjI4MThmYzEifQ==",
    )

    run["parameters"] = current_parameters

    # Создаем и обучаем модель XGBoost
    xgb_model = xgb.XGBRegressor(
        objective="reg:squarederror",
        n_estimators=current_parameters["n_estimators"],
        learning_rate=current_parameters["learning_rate"],
        max_depth=current_parameters["max_depth"]
    )
    
    xgb_model.fit(X_train, y_train)

    # Оцениваем качество модели на тестовой выборке
    y_pred = xgb_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    # Логируем метрики
    run['mse'] = mse

    # Сохраняем модель
    joblib.dump(xgb_model, 'xgb_model.pkl')
    run['model'].upload('xgb_model.pkl')

    # Завершение эксперимента
    run.stop()

[neptune] [warning] NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/svterskov/LSML2/e/LSML2-9


C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\neptune\internal\utils\git.py:71: UserWarning: GitPython could not be initialized
  warnings.warn("GitPython could not be initialized")


[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 2 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 2 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/svterskov/LSML2/e/LSML2-9/metadata


C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\neptune\internal\utils\git.py:71: UserWarning: GitPython could not be initialized
  warnings.warn("GitPython could not be initialized")


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/svterskov/LSML2/e/LSML2-10
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 2 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 2 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/svterskov/LSML2/e/LSML2-10/metadata


C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\neptune\internal\utils\git.py:71: UserWarning: GitPython could not be initialized
  warnings.warn("GitPython could not be initialized")


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/svterskov/LSML2/e/LSML2-11
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 2 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 2 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/svterskov/LSML2/e/LSML2-11/metadata


## Сохранение векторов TF-IDF и MultiLabelBinarizer для использования в сервисе

In [10]:
joblib.dump(vectorizer, 'vectorizer.pkl')
joblib.dump(mlb, 'mlb.pkl')

['mlb.pkl']

# Файлы python для сервиса модели

## Model.py

In [20]:
%%writefile model.py

import joblib
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer

class Model:
    def __init__(self):
        self.model = joblib.load('xgb_model.pkl')
        self.vectorizer = joblib.load('vectorizer.pkl')
        self.mlb = joblib.load('mlb.pkl')

    def predict(self, text, rubrics):
        rubrics_vector = self.mlb.transform([rubrics])[0]
        text_vector = self.vectorizer.transform([text]).toarray()[0]
        input_vector = list(text_vector) + list(rubrics_vector)
        return self.model.predict([input_vector])[0]

Writing model.py


## App.py

In [21]:
%%writefile app.py

from flask import Flask, request, render_template, jsonify
from model import Model

app = Flask(__name__)
model = Model()

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    text = data['text']
    rubrics = data['rubrics']
    prediction = round(model.predict(text, rubrics))
    
    return jsonify({'prediction': prediction})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

Writing app.py
